In [1]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content="In the realm of code where wizards dwell,\nThere lies a mystical spell,\nRecursion, a concept so profound,\nIn loops and functions, it is found.\n\nLike a mirror reflecting its own reflection,\nRecursion calls itself without rejection,\nA function that calls upon itself,\nCreating a loop in a magical spell.\n\nThrough layers of recursion, we traverse,\nLike a journey through a universe,\nEach call leaving a breadcrumb trail,\nUntil the base case lifts the veil.\n\nLike a Russian nesting doll so grand,\nRecursive functions in code expand,\nIn infinite loops or finite grace,\nRecursion weaves its intricate lace.\n\nSo remember, programmer bold and true,\nRecursion holds a secret view,\nA loop within a loop, a code's delight,\nIn the realm of programming, shining bright.", role='assistant', function_call=None, tool_calls=None)
